### 1. 算法简介

- **提出背景**：经典的 LDA 公式要求$S_w$是可逆的，但对于样本维度远大于样本数量的欠采样问题，由于$S_w$都是奇异的，经典的 LDA 失效，而NLDA(零空间线性判别分析)和OLDA(正交线性判别分析)已经被提出以克服这个问题。


- **两种的算法介绍**：
  - NLDA 旨在在类内散布矩阵的零空间中最大化类间距离。
  - OLDA 通过同时对散布矩阵进行对角化来计算一组正交判别向量。

- **研究成果**：
  - 本文结果表明，在许多涉及高维数据的应用中，NLDA 在温和条件下等同于 OLDA。
  - 本文将正则化应用于OLDA，称为 ROLDA，并且提出了一种高效算法来估计 ROLDA 中的正则化值。

### 2. 算法流程

#### 2.1. NLDA (零空间LDA)

Chen等人在2000年提出了零空间LDA（NLDA），用于降维，其中类间距离在类内散布矩阵的零空间中被最大化。这种算法背后的基本思想是，如果$S_b$在该方向的投影非零，则$S_{w}$的零空间可能包含显著的判别信息（Chen等，2000; Lu等，2003）。因此，隐含地克服了奇异性问题。通过解决以下优化问题可以计算NLDA的最优变换：
$$G=\operatorname{argmax}_{G^TS_wG=0}\operatorname{trace}(G^TS_bG).$$
(7)
最优$G$的计算涉及到$S_w$的零空间的计算，这对于高维数据来说可能是庞大的。事实上，$S_w$的零空间的维度至少为$m+k-n$，其中$m$是数据维度，$k$是类别数，$n$是样本大小。在Chen等人的2000年的研究中，使用了像素分组方法来提取几何特征并降低样本的维度，然后在新的特征空间中应用NLDA。Huang等人在2002年改进了Chen等人2000年的算法，首先移除了总散布矩阵$S_t$的零空间。这是基于观察到$S_{t}$的零空间是$S_b$和$S_w$的零空间的交集，因为$S_t=S_w+S_b$。
我们可以按如下方式有效地移除$S_t$的零空间。设$H_t=U\Sigma V^T$是$H_t$的奇异值分解（SVD）（Golub和Van Loan，1996），其中$H_t$在方程（3）中定义，$U$和$V$是正交的，
$$\Sigma=\left(\begin{array}{cc}\Sigma_t&0\\0&0\end{array}\right),$$
$\Sigma_t\in\mathbb{R}^{t\times t}$是对角的，对角线元素按非递增顺序排列，且$t=$rank$(S_t).$
然后
$$\left.S_t=H_tH_t^T=U\Sigma V^TV\Sigma^TU^T=U\Sigma\Sigma^TU^T=U\left(\begin{array}{cc}\Sigma_t^2&0\\0&0\end{array}\right.\right)U^T.$$
(8)
$\operatorname{Let}U=(U_1,U_2)$是$U$的一个划分，其中$U_1\in\mathbb{R}^{m\times t}$，$U_2\in\mathbb{R}^{m\times(m-t)}$。然后可以通过将数据投影到由$U_1$的列跨度的子空间中来移除$S_t$的零空间。设$\tilde{S}_b,\tilde{S}_w$和$\tilde{S}_t$为移除$S_t$的零空间后的散布矩阵，即
$$\tilde{S}_b=U_1^TS_bU_1,\:\tilde{S}_w=U_1^TS_wU_1,\:\mathrm{and}\:\tilde{S}_t=U_1^TS_tU_1.$$
注意只有$U_1$参与了投影。我们因此可以在$H_t$上应用降低的SVD计算（Golub和Van Loan，1996），时间复杂度为$O(mm^2)$，而不是$O(m^2n)$。当数据维度$m$远大于样本大小$n$时，这导致了计算成本的大幅降低。
通过计算得到的$U_1$，NLDA的最优变换由$G=U_1N$给出，其中$N$
通过解决以下优化问题获得：
$$N=\operatorname{argmax}_{N^T\tilde{S}_wN=0}\operatorname{trace}(N^T\tilde{S}_bN).$$
$$(9)$$
也就是说，$N$的列位于$\tilde{S}_w$的零空间中，同时最大化$trace(N^T\tilde{S}_bN)$。
设$W$是矩阵，使得$W$的列跨越$\tilde{S}_w$的零空间。然后$N=WM$，对于某个待确定的矩阵$M$。由于方程(9)的约束条件使用$N=WM$对任何$M$都成立，最优$M$可以通过最大化
$$\mathrm{trace}(M^TW^T\tilde{S}_bWM).$$
通过在$M$上施加正交约束（Huang等，2002），最优$M$由$W^T\tilde{S}_bW$对应的非零特征值的特征向量给出。通过计算得到的$U_1,W$和$M$，NLDA的最优变换由
$$G=U_1WM.$$
给出。在（Huang等人，2002）中，通过$\tilde{S}_w$的特征分解计算出矩阵$W$。
具体来说，设
$$\tilde{S}_w=[W,\tilde{W}]\left(\begin{array}{cc}0&0\\0&\Delta_w\end{array}\right)[W,\tilde{W}]^T$$
是它的特征分解，其中$[W,\tilde{W}]$是正交的，$\Delta_\mathrm{w}$是对角的，对角线上的元素为正。然后$W$构成了Š$_w$的零空间。NLDA算法的伪代码在$\mathbf{Algorithm}1$中给出。

$\textbf{Algorithm 1: NLDA（零空间LDA）}$

$\textbf{输入：}\:\text{数据矩阵 }$

$ 输出：变换矩阵G $

1. 形成矩阵 $H_t=\frac{1}{\sqrt{n}}(A-ce^T)$；
2. 计算$H_t$的降维SVD为$H_{\underline{t}}=U_1\Sigma_tV_1^T;$ 
3. 形成矩阵$\tilde{S}_b=U_1^TS_bU_1$ 和 $\tilde{S}_w=U_1^TS_wU_1;$ 
4. 通过特征分解计算$\tilde{S}_w$的零空间，$\tilde{{S}}_{w}$的 $W$;
5. 构建矩阵 $M$，包括 $W^T\tilde{S}_bW$ 的顶部特征向量；
6. $G\leftarrow U_{1}WM$.

#### 2.2. OLDA (正交化LDA)

正交LDA（Orthogonal LDA, OLDA）于2005年由Ye提出，作为经典LDA的一个扩展。在OLDA中，判别向量彼此正交。此外，即使当所有散布矩阵都是奇异的时候，OLDA仍然适用，从而克服了奇异性问题。它已在多种应用中成功应用，包括文档分类、面部识别和基因表达数据分类。OLDA中的最优变换可以通过解决以下优化问题来计算：
$$G=\operatorname{argmax}_{G\in\mathbb{R}^{m\times\ell} : G^TG=I_\ell}\operatorname{trace}\left((G^TS_tG)^+G^TS_bG\right),$$
其中 $M^+$ 表示矩阵 $M$ 的伪逆。约束条件中施加了正交性。OLDA的最优变换计算基于三个散布矩阵的同时对角化，如下所述：
从方程 (8) 知，$U_2$ 位于 $S_b$ 和 $S_w$ 的零空间中。因此：
$$
U^TS_bU=\left(\begin{array}{cc}U_1^TS_bU_1&0\\0&0\end{array}\right),\quad U^TS_wU=\left(\begin{array}{cc}U_1^TS_wU_1&0\\0&0\end{array}\right).
$$
记 $B = \Sigma_t^{-1}U_1^TH_b$，设 $B = P\tilde{\Sigma}Q^T$ 为 $B$ 的奇异值分解（SVD），其中 $P$ 和 $Q$ 是正交的，$\tilde{\Sigma}$ 是对角的。定义矩阵 $X$ 如下：
$$
X=U\left(\begin{array}{cc}\Sigma_t^{-1}P&0\\0&I_{m-t}\end{array}\right).
$$
可以证明（Ye, 2005），$X$ 同时对角化了 $S_b$、$S_w$ 和 $S_t$，即：
$$
X^TS_bX=D_b,\:X^TS_wX=D_w,\:\mathrm{and}\:X^TS_tX=D_t,
$$
其中 $D_b$、$D_w$ 和 $D_t$ 都是对角的，$D_b$ 的对角元素按非递增顺序排列。Ye (2005) 的主要结果显示，可以通过对 $X$ 列的正交化计算 OLDA 的最优变换，如下定理所述：

**定理 4.1** 设 $X$ 为在方程 (12) 中定义的矩阵，并让 $X_q$ 为 $X$ 的前 $q$ 列组成的矩阵，其中 $q=\text{rank}(S_b)$。设 $X_q = QR$ 为 $X_q$ 的 QR 分解，其中 $Q$ 的列向量是正交规范的，$R$ 是上三角的。则 $G=Q$ 解决了方程 (10) 中的优化问题。
根据定理 4.1，只有 $X$ 的前 $q$ 列用于计算最优的 $G$。从方程 (12) 知，$X_q$ 由下式给出：
$$
X_q = U_1\Sigma_t^{-1}P_q,
$$
其中 $P_q$ 是矩阵 $P$ 的前 $q$ 列。我们可以观察到 $U_1$ 对应于移除 $S_t$ 的零空间，如同在 NLDA 中，而 $\Sigma_t^{-1}P_q$ 是当将经典的 ULDA 应用于通过 $U_1$ 投影得到的中间（降维后的）空间时的最优变换。因此，OLDA 算法可以分解为三个步骤：（1）移除 $S_t$ 的零空间；（2）作为中间步骤应用经典 ULDA，因为降低的总散布是非奇异的；（3）应用一个正交化步骤到变换中，这对应于定理 4.1 中的 $X_q$ 的 QR 分解。OLDA 算法的伪代码在算法 2 中给出。

$\textbf{Algorithm 2: OLDA（正交LDA）}$

**输入**：A

**输出**：变换矩阵 G

1. 组成$U_1,\sum_t$和$P$
2. $X_q\leftarrow U_1\Sigma_t^{-1}P_q,$其中$q=\mathrm{rank}(S_b)$
3. 计算 $X_q$ 的 QR 分解，表示为 $X_q = QR$。
4. $G\leftarrow Q$


#### 2.3. ROLDA 正则化正交LDA

回想一下，OLDA涉及总散布矩阵的伪逆，其估计可能不太可靠，特别是对于维度数超过样本大小的欠采样数据。在这种情况下，参数估计可能非常不稳定，导致高方差。通过采用正则化方法，人们试图通过调节这种偏差-方差权衡来改善估计（Friedman, 1989）。我们通过在总散布矩阵的对角元素上添加一个常数 $\lambda$ 来将正则化技术应用于OLDA，这里 $\lambda>0$ 被称为正则化参数。该算法被称为正则化OLDA（ROLDA）。ROLDA的最优变换 $G^r$ 可以通过解决以下优化问题来计算：
$$ G^r = \operatorname{argmax}_{G \in \mathbb{R}^{m \times \ell} : G^T G = I_\ell} \operatorname{trace}\left(\left(G^T (S_t + \lambda I_m) G\right)^+ G^T S_b G\right). $$
最优的 $G^r$ 可以通过解决一个特征值问题来计算，如下定理中总结（证明遵循Ye, 2005中的定理3.1，因此省略）：


**定理 6.1** 设 $X_q^r$ 是由矩阵 
$$ (S_t + \lambda I_m)^{-1} S_b $$
对应非零特征值的前 $q$ 个特征向量组成的矩阵，其中 $q=\text{rank}(S_b)$。设 $X_q^r = QR$ 为 $X_q^r$ 的QR分解，其中 $Q$ 的列向量是正交规范的，$R$ 是上三角的。然后 $G = Q$ 解决了方程 (19) 中的优化问题。
定理6.1意味着ROLDA中涉及的主要计算是矩阵 
$$ (S_t + \lambda I_m)^{-1} S_b $$
的特征分解。直接形成矩阵对于高维数据来说是昂贵的，因为它的大小是 $m \times m$。以下，我们提出了一种高效计算特征分解的方法。记
$$ B^r = (\Sigma_t^2 + \lambda I_t)^{-1/2} U_1^T H_b $$
并设
$$ B^r = P^r \tilde{\Sigma}^r (Q^r)^T $$
为 $B^r$ 的SVD。从方程(8)和(11)我们有
$$
\begin{aligned}
(S_{t} + \lambda I_{m})^{-1} S_{b} &= U \left(\begin{array}{ccc}
(\Sigma_t^2 + \lambda I_t)^{-1} & 0 \\
0 & \lambda^{-1} I_{m-t}
\end{array}\right) U^T U \left(\begin{array}{ccc}
U_1^T S_b U_1 & 0 \\
0 & 0
\end{array}\right) U^T \\
&= U \left(\begin{array}{ccc}
(\Sigma_t^2 + \lambda I_t)^{-1/2} B^r (B^r)^T (\Sigma_t^2 + \lambda I_t)^{1/2} & 0 \\
0 & 0
\end{array}\right) U^T.
\end{aligned}
$$
结果表明，
$$ U_1 (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r $$
的列向量形成了对应于前 $q$ 个非零特征值的 $(S_t + \lambda I_m)^{-1} S_b$ 的特征向量，其中 $P_q^r$ 表示 $P^r$ 的前 $q$ 列。即，

定理 6.1 中的 $X_q^r$ 由
$$ X_q^r = U_1 (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r $$
给出。
ROLDA 算法的伪代码在算法 4 中给出。ROLDA中的计算可以分解为两个组成部分：第一部分涉及高维度但与 $\lambda$ 无关的矩阵 $U_1 \in \mathbb{R}^{m \times t}$，第二部分涉及低维度的矩阵 
$$ (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r \in \mathbb{R}^{t \times q} $$
当我们使用交叉验证从候选集中搜索最优的 $\lambda$ 时，我们只重复第二部分涉及的计算，从而使模型选择的计算成本很小。
具体来说，设
$$ \Lambda = \{\lambda_1, \cdots, \lambda_{|\Lambda|}\} $$
为正则化参数 $\lambda$ 的候选集，其中 $|\Lambda|$ 表示候选集Λ的大小。我们应用 $ \nu $-折交叉验证进行模型选择（我们的实验中选择 $ \nu=5 $），其中数据被分为 $ \nu $ 个（大致）相等大小的子集。所有子集是相互排斥的，在第 $i$ 轮中，第 $i$ 个子集被保留用于测试，所有其他子集用于训练。对于每个 $\lambda_j (j=1, \cdots, |\Lambda|)$，我们计算交叉验证准确度 Accu($j$)，定义为所有轮次的准确度的平均值。最优的正则化值 $\lambda_{j^*}$ 是具有
$$ j^* = \arg\max_j \text{Accu}(j) $$
的那个。计算准确度时使用的是 $K$-最近邻算法，其中 $K=1$，称为 1-NN。在ROLDA中的模型选择程序的伪代码在算法 5 中给出。注意，我们应用 QR 分解到
$$ (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r \in \mathbb{R}^{t \times q} $$
而不是
$$ X_q^r = U_1 (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r \in \mathbb{R}^{m \times q}, $$
如定理 6.1 所做的那样，因为 $U_{1}$ 的列向量是正交规范的。

$\textbf{Algorithm 4:  ROLDA (正则化正交LDA)}$

输入：数据矩阵 $A$ 和正则化值 $\lambda$
输出：变换矩阵 $G^r$
1. 计算 $U_1, \Sigma_t$ 和 $P_q^r$，其中 $q=\text{rank}(S_b)$;
2. $X_{q}^{r} \leftarrow U_1 (\Sigma_t^2 + \lambda I_t)^{-1/2} P_q^r$;
3. 计算 $X_q^r$ 的 QR 分解为 $X_q^r = QR$;
4. $G^{r} \leftarrow Q$.

### 3. 摘译

降维是许多应用中的一个重要预处理步骤。线性判别分析 (LDA) 是一种经典的统计方法，用于监督降维。它旨在最大化类间距离与类内距离的比率，从而最大化类别区分能力。LDA 在许多应用中得到了广泛使用。然而，经典的 LDA 公式要求散布矩阵是非奇异的。对于样本维度远大于样本数量的欠采样问题，所有的散布矩阵都是奇异的，经典的 LDA 失效。许多扩展方法，包括零空间 LDA (NLDA) 和正交 LDA (OLDA)，已经被提出以克服这个问题。NLDA 旨在在类内散布矩阵的零空间中最大化类间距离，而 OLDA 通过同时对散布矩阵进行对角化来计算一组正交判别向量。它们在各种应用中都取得了成功。
在本文中，我们对 NLDA 和 OLDA 进行了计算和理论分析。我们的主要结果表明，在许多涉及高维数据的应用中，NLDA 在温和条件下等同于 OLDA。我们在各种类型的数据上进行了广泛的实验，结果与我们的理论分析一致。我们进一步将正则化应用于 OLDA，算法称为正则化 OLDA (简称 ROLDA)。我们提出了一种高效算法来估计 ROLDA 中的正则化值。对分类的比较研究表明，ROLDA 在与 OLDA 的竞争中表现得非常出色。这证实了 ROLDA 中正则化的有效性。